In [ ]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
WARNING  /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/plotly/graph_objects/__init__.py:288:              warnings.py:109
           if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):                                                           
                                                                                                                                       
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
%load_ext autoreload
%autoreload 2

WARNING  /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py:121:              warnings.py:109
         see the module's documentation for alternative uses                                                                           
           from imp import reload                                                                                                      
                                                                                                                                       


In [ ]:
view = "texas_charges"

# LIM = 100000
enc = catalog.load(f'{view}.wrk.idx_table').sample()#[:LIM]
trn = catalog.load(f'{view}.trn.table')
attrs = trn['idx'].get_attributes()
len(enc)

9251363


In [ ]:
from pasteur.extras.synth.math import expand_table, calc_marginal, AttrSelector, AttrSelectors, get_dtype

cols, cols_noncommon, domains = expand_table(attrs, enc)
domains.keys()

dict_keys(['revenue_code', 'hcpcs_qualifier', 'hcpcs_procedure_code', 'modifier_1', 'modifier_2', 'modifier_3', 'modifier_4', 'unit_measurement_code', 'units_of_service', 'unit_rate', 'chrgs_line_item', 'chrgs_non_cov'])


In [ ]:
tst = ['unit_measurement_code', 'unit_measurement_code', 'unit_measurement_code', 'unit_measurement_code','unit_measurement_code','unit_measurement_code']
# tst = ['unit_measurement_code', 'unit_measurement_code', 'unit_measurement_code']

In [ ]:
x = AttrSelector(tst[0], 0, {tst[0]: 0})
p = {str(i): AttrSelector(col, 0, {col: 0}) for i, col in enumerate(tst[1:])}

%timeit calc_marginal(cols, cols_noncommon, domains, x, p)

90.3 ms ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
import numpy as np

%timeit np.histogramdd(enc[tst].to_numpy(), [domains[col][0] for col in tst])

682 ms ± 2.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%load_ext line_profiler

In [ ]:

def calc_marginal2(
    cols: dict[str, list[np.ndarray]],
    cols_noncommon: dict[str, list[np.ndarray]],
    domains: dict[str, list[int]],
    x: AttrSelector,
    p: AttrSelectors,
    partial: bool = False,
    zero_fill: float | None = None,
):
    """Calculates the 1 way and 2 way marginals between the subsection of the
    hierarchical attribute x and the attributes p(arents)."""

    # Find integer dtype based on domain
    p_dom = 1
    for attr in p.values():
        common = attr.common
        l_dom = 1
        for i, (n, h) in enumerate(attr.cols.items()):
            l_dom *= domains[n][h] - common
        p_dom *= l_dom + common
    x_dom = 1
    for i, (n, h) in enumerate(x.cols.items()):
        x_dom *= domains[n][h] - x.common
    x_dom += x.common

    dtype = get_dtype(p_dom * x_dom)

    n = len(next(iter(cols.values()))[0])
    _sum_nd = np.zeros((n,), dtype=dtype)
    _tmp_nd = np.zeros((n,), dtype=dtype)

    # Handle parents
    mul = 1
    for attr_name, attr in p.items():
        common = attr.common
        l_mul = 1
        p_partial = partial and attr_name == x.name
        for i, (n, h) in enumerate(attr.cols.items()):
            if common == 0 or i == 0:
                np.multiply(cols[n][h], mul * l_mul, out=_tmp_nd, dtype=dtype)
            else:
                np.multiply(cols_noncommon[n][h], mul * l_mul, out=_tmp_nd, dtype=dtype)

            np.add(_sum_nd, _tmp_nd, out=_sum_nd, dtype=dtype)
            l_mul *= domains[n][h] - common

        if p_partial:
            mul *= l_mul
        else:
            mul *= l_mul + common

    # Handle x
    common = x.common
    for i, (n, h) in enumerate(x.cols.items()):
        if common == 0 or (i == 0 and not partial):
            np.multiply(cols[n][h], mul, out=_tmp_nd, dtype=dtype)
        else:
            np.multiply(cols_noncommon[n][h], mul, out=_tmp_nd, dtype=dtype)

        np.add(_sum_nd, _tmp_nd, out=_sum_nd, dtype=dtype)
        mul *= domains[n][h] - common

    # Keep only non-common items if there is a parent to source the others
    if partial:
        n = next(iter(x.cols))
        _sum_nd = _sum_nd[cols[n][0] >= common]
        x_dom = x_dom - x.common

    return np.bincount(_sum_nd, minlength=p_dom * x_dom)#.reshape(x_dom, p_dom)
    # return np.zeros()

    # j_mar = margin
    # x_mar = np.sum(margin, axis=1)
    # p_mar = np.sum(margin, axis=0)

    # return j_mar, x_mar, p_mar

%timeit calc_marginal2(cols, cols_noncommon, domains, x, p)
# %lprun -f calc_marginal2 calc_marginal2(cols, cols_noncommon, domains, x, p)
b = calc_marginal2(cols, cols_noncommon, domains, x, p)

77.1 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# import jax.numpy as jnp
# from jax import grad, jit, vmap, default_device, devices
# from jax import random, ops
# from typing import NamedTuple
# import jax.lax as lax


# class JaxSelector(NamedTuple):
#     common: int
#     cols: list[tuple[int, jnp.ndarray]]


# JaxSelectors = list[JaxSelector]


# def collapse_marginal(x: JaxSelector, P: JaxSelectors):
#     # Handle parents
#     sum_arr = None
#     mul = 1
#     for p in P:
#         common = p.common
#         l_mul = 1
#         for dom, col in p.cols:
#             tmp_arr = col * (mul * l_mul)
#             if sum_arr is None:
#                 sum_arr = tmp_arr
#             else:
#                 sum_arr += tmp_arr
#             l_mul *= dom - common

#     # Handle x
#     assert sum_arr is not None
#     common = x.common
#     for dom, col in x.cols:
#         sum_arr += dom * mul
#         mul *= dom - common

#     return sum_arr

In [ ]:
# def count(sum_arr, length: int):
#     # return jnp.bincount(sum_arr, length=length)
#     # return lax.broadcast(jnp.uint32(0), [length]).at[sum_arr].add(jnp.uint8(1))
#     def f(carry, x):
#         return carry, x
    
#     return lax.scan(f, lax.broadcast(jnp.uint16(0), [length]), sum_arr)
#     # op = lax.broadcast(jnp.uint16(0), [length])
#     # sc = lax.expand_dims(sum_arr, [1])
#     # up = lax.broadcast(jnp.uint16(1), [sc.shape[0]])
#     # return lax.scatter_add(op, sc, up, lax.ScatterDimensionNumbers(tuple(), (0,), (0,)), mode="promise_in_bounds")

# def fun():
#     x = JaxSelector(0, [(domains[tst[0]][0], jnp.array(cols[tst[0]][0]))])
#     P = []
#     for col in tst[1:]:
#         P.append(JaxSelector(0, [(domains[col][0], jnp.array(cols[col][0]))]))
    
#     p_dom = 1
#     for p in P:
#         common = p.common
#         l_dom = 1
#         for dom, _ in p.cols:
#             l_dom *= dom - common
#         p_dom *= l_dom + common
#     x_dom = 1
#     for dom, _ in x.cols:
#         x_dom *= dom - x.common
#     x_dom += x.common
#     length = p_dom * x_dom
#     print(length)

#     try:
#         cmp = jit(collapse_marginal).lower(x, P)
#         # print(cmp.as_text())
#         cmp = cmp.compile()
#         # print(cmp.as_text())

#         print(jit(count, static_argnums=1).lower(cmp(x, P), length).as_text())
#         print(count(cmp(x, P), length)[:10])
#     except Exception as e:
#         print(e)
#         return

#     TIMES = 10
#     def xn():
#         out = count(cmp(x, P), length)

#         for i in range(TIMES-1):
#             out += count(cmp(x, P), length)
        
#         out.block_until_ready()

#     # %timeit xn()
#     %timeit count(cmp(x, P), length).block_until_ready()
#     %timeit xn()


# for cuda in (False,): #True):
#     d = devices('cuda' if cuda else 'cpu')[0]
#     with default_device(d):
#         fun()


In [ ]:
Op = tuple[int, np.ndarray]

from pasteur.marginal.native import marginal

def calc_marginal_native(
    cols: dict[str, list[np.ndarray]],
    cols_noncommon: dict[str, list[np.ndarray]],
    domains: dict[str, list[int]],
    x: AttrSelector,
    p: AttrSelectors,
    partial: bool = False,
    zero_fill: float | None = None,
    out: np.ndarray | None = None
):
    """Calculates the 1 way and 2 way marginals between the subsection of the
    hierarchical attribute x and the attributes p(arents)."""

    # Keep only non-common items if there is a parent to source the others
    # if partial:
    #     n = next(iter(x.cols))
    #     mask = cols[n][0] >= x.common
    #     x_dom = x_dom - x.common


    # Handle parents
    ops: list[Op] = []
    mul = 1
    for attr_name, attr in p.items():
        common = attr.common
        l_mul = 1
        p_partial = partial and attr_name == x.name
        for i, (n, h) in enumerate(attr.cols.items()):
            if common == 0 or i == 0:
                ops.append((mul*l_mul, cols[n][h]))
            else:
                ops.append((mul*l_mul, cols_noncommon[n][h]))

            l_mul *= domains[n][h] - common

        if p_partial:
            mul *= l_mul
        else:
            mul *= l_mul + common
    p_dom = mul

    # Handle x
    common = x.common
    l_mul = 1
    for i, (n, h) in enumerate(x.cols.items()):
        if common == 0 or (i == 0 and not partial):
            ops.append((mul*l_mul, cols[n][h]))
        else:
            ops.append((mul*l_mul, cols_noncommon[n][h]))

        l_mul *= domains[n][h] - common
    
    if not partial:
        l_mul += common
    x_dom = l_mul
    dom = mul * l_mul

    if out is None:
        out = np.zeros((dom, ), dtype=np.uint32)
    else:
        out = out.reshape((-1,))

    marginal(dom, out, ops)

    return out.reshape((x_dom, p_dom))

%timeit calc_marginal_native(cols, cols_noncommon, domains, x, p)
# %lprun -f calc_marginal2 calc_marginal2(cols, cols_noncommon, domains, x, p)
a = calc_marginal_native(cols, cols_noncommon, domains, x, p)

18.4 ms ± 26.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
len(a)

6


In [ ]:
len(b)

46656


In [ ]:
b[a.reshape(-1) != b]

array([8532670])


In [ ]:
a.reshape(-1)[a.reshape(-1) != b]

array([8532667], dtype=uint32)


In [ ]:
a[a != 0]

array([    210,  718467,       4,      12, 8532667], dtype=uint32)


In [ ]:
b[b != 0]

array([    210,  718467,       4,      12, 8532670])
